### 1 - Import libs

**Execute only**

In [1]:
import time
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap

# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

Styles()

/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**DEFINE YOUR INPUT VARIABLES**

In [10]:
# Area of Interest
#country = 'Kenya'   # country case
#aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected
aoi = ee.FeatureCollection("users/Erith7/Sucumbios")
# Grid Size (in degrees)
grid_size = 0.01

# define an output file name for CEO 
# don't forget the .csv extention
# IF NOT DEFINED; will go into module_results
 #outfile = "sbae.csv"

### 3 - Generate GRID and POINTS

**Execute only**

In [11]:
# Generate the grid
grid, grid_fc = h.generate_grid(aoi, grid_size, grid_size, 0, 0)

# get a random point over each grid cell
randomInCell = grid_fc.map(h.get_random_point).filterBounds(aoi)

# get the centroid of each gridcell
centerInCell = grid_fc.map(h.get_center_point).filterBounds(aoi)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 1000,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

### 4 - Display in map

**Execute only**

In [13]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi,color=1,width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples.draw(color="black", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')
Map.addLayer(ee.Image().byte().paint(featureCollection=grid,color=1,width=3), {"palette": "black"}, 'Grid')
Map.addLayer(centerInCell.draw(color="white", pointRadius=2), {}, '1 Center Sample per gridcell (Systematic)')
Map.addLayer(randomInCell.draw(color="purple", pointRadius=2), {}, '1 Random Sample per gridcell (Systematic random)')
#
Map

SepalMap(center=[-0.041799613075468256, -77.15800826163783], controls=(ZoomControl(options=['position', 'zoom_…

### 5 - Add auxillary data from global datasets

**Execute only**

In [9]:
point_id = 'point_id'
point_with_global_products = h.sample_global_products(centerInCell, point_id)
point_with_global_products

,LON,LAT,PLOTID,geometry,esa_lc20,gfc_gain,gfc_loss,gfc_tc00,gfc_year,point_id,...,tmf_1995,tmf_2000,tmf_2005,tmf_2010,tmf_2015,tmf_2020,tmf_def_yr,tmf_deg_yr,tmf_main_cl,tmf_subtypes
0,-77.214998,-0.080188,220,POINT (-77.21500 -0.08019),10,0,0,98,None,220,...,1,1,2,2,2,2,0,2001,20,26
1,-77.214998,0.019812,221,POINT (-77.21500 0.01981),10,0,0,99,None,221,...,1,1,1,1,1,2,0,2016,20,22
2,-77.114998,-0.080188,241,POINT (-77.11500 -0.08019),10,0,0,100,None,241,...,1,1,1,1,1,1,0,0,10,10
3,-77.114998,0.019812,242,POINT (-77.11500 0.01981),10,0,0,97,None,242,...,1,1,1,1,1,1,0,0,10,10
